In [3]:
# Header
from nupack import *
import numpy as np
config.parallelism = True

Model1 = Model(material='dna04', ensemble='some-nupack3', celsius=29, sodium=0.1, magnesium=0.0125) #Define model used for NUPACK calculations

In [4]:
# Import strands
Strands_list = []
Interfering = open("Designs.txt")
for line in Interfering:
    Strands_list.append(line.rstrip())
Interfering.close()
Nlength = 35 #Number of bases of each interfering strand (fixed in our case)

In [5]:
# Define strands/complexes and tasks 
Invader_Strand = Strand('CGGAATAAGGCAAGATAAGAACAGATACATAGGACGGATAGAGAA', name='Invader')
Interfering_free_energy_list = []
results = open("Results.txt", 'a')
for i in range(len(Strands_list)):
    Interfering_Strand = Strand(Strands_list[i], name ='Interfering_Strand')
    Interfering_Invader_Complex = Complex([Interfering_Strand, Invader_Strand], name='Interfering_Invader_Complex')
    Interfering_Complex = Complex([Interfering_Strand], name='Interfering_Complex')
    Invader_Complex = Complex([Invader_Strand], name='Invader_Complex')
    InteractionTube = Tube({Invader_Strand: 20e-9, Interfering_Strand: 1e-6}, complexes=SetSpec(include=[Interfering_Invader_Complex, Interfering_Complex, Invader_Complex]), name='InteractionTube')
    my_results = tube_analysis([InteractionTube], model=Model1, compute=['mfe'])
    Interfering_free_energy_list.append(str(my_results[Interfering_Complex].free_energy))
    results.write(str(Interfering_Strand) + '\n' +str(my_results[Interfering_Invader_Complex].free_energy) +'\n'+ str(my_results[Interfering_Invader_Complex].mfe[0].structure) +'\n'+'\n')
results.close()

In [6]:
# Calculate Energies
Results_list = []

results = open("Results.txt")
for line in results:
    Results_list.append(line.rstrip())
results.close()

Energies_list=[]

for i in range(len(Results_list[1::4])):
    Energies_list.append(float(Results_list[1::4][i])-float(Interfering_free_energy_list[i])-my_results[Invader_Complex].free_energy)

Complexes_ddG = open("Complex_ddG.txt", 'a')

for i in range(len(Energies_list)):
    Complexes_ddG.write(str("{:.2f}".format(Energies_list[i]))+'\n'),
Complexes_ddG.close()

In [7]:
# BM features
Bound_BM_Invader = open("Bound_BM_Invader.txt", 'a')
Binding_Blocks_BM= open("Binding_Blocks_BM.txt", 'a')
Longest_Bound_BM = open("Longest_Bound_BM.txt", 'a')
Longest_Unbound_BM = open("Longest_Unbound_BM.txt", 'a')

for j in range(0,len(Strands_list)):
    bound_BM = 0
    Interfering_Strand = Strand(Strands_list[j], name ='Interfering_Strand')
    probability_matrix = pairs(strands=[Interfering_Strand, Invader_Strand], model=Model1)
    probability_array=probability_matrix.to_array()
    rev_struc_body = str(Results_list[2::4][j][Nlength+1:Nlength+26][::-1])
    partition =[]
    bound_unbound = (''.join(x + ('' if x == nxt else ',') for x, nxt in zip(rev_struc_body, rev_struc_body[1:] + rev_struc_body[-1])))
    partition = bound_unbound.split(',')
    splitted = []
    for i in range(len(partition)):
        if '.' in partition[i]:
            splitted.append(float(len(partition[i])))
        else:
            splitted.append(str(len(partition[i])))          

    ###############################################################################
    # Code for number of binding blocks
    bindingblocks = 0
    for i in range(len(splitted)):
        if type(splitted[i]) == str:
            bindingblocks+=1
    Binding_Blocks_BM.write(str("{:.2f}".format((bindingblocks))+"\n"))
    ##############################################################################
    ##############################################################################
    # Code for longest stretch of bound bases
    longestbound = 0
    boundstretch = 0
    position = 0
    bound_stretches=[]
    for i in range(len(splitted)):
        if type(splitted[i]) == str:
            bound_stretches.append(float(splitted[i]))
    if len(bound_stretches) > 0:
        for i in range(0,splitted.index(str(int(max(bound_stretches))))):
            position += int(splitted[i])
        for m in range(Nlength+25-int(position+max(bound_stretches)), Nlength+25-int(position)): 
        #It can make sense to use the bases left and/or right of the stretch, too. This would be inbluded by +/-1 in the range.
            boundstretch += (1-probability_array[m][m])
        Longest_Bound_BM.write(str("{:.2f}".format(boundstretch)+"\n"))
    else:
        Longest_Bound_BM.write(str("{:.2f}".format((0))+"\n"))
    #############################################################################
    
    ##############################################################################
    # Code for longest stretch of unbound bases
    longestunbound = 0
    unboundstretch = 0
    position = 0
    unbound_stretches=[]
    for i in range(len(splitted)):
        if type(splitted[i]) == float:
            unbound_stretches.append(float(splitted[i]))
    if len(unbound_stretches) > 0:
        for i in range(0,splitted.index(max(unbound_stretches))):
            position += float(splitted[i])
        for m in range(Nlength+25-int(position+max(unbound_stretches)), Nlength+25-int(position)):
        #It can make sense to use the bases left and/or right of the stretch, too. This would be inbluded by +/-1 in the range.
            unboundstretch += probability_array[m][m]
        Longest_Unbound_BM.write(str("{:.2f}".format(unboundstretch)+"\n"))
    else:
        Longest_Unbound_BM.write(str("{:.2f}".format((0))+"\n"))
    #############################################################################
    bound = 0
    for m in range(Nlength,Nlength+25):
        bound += (1-probability_array[m][m])
    bound_BM += bound
    Bound_BM_Invader.write(str("{:.2f}".format((bound_BM))+"\n"))
Bound_BM_Invader.close()
Binding_Blocks_BM.close()
Longest_Bound_BM.close()
Longest_Unbound_BM.close()